# classification of one card

In [ ]:
# dictonary from labels(str) to int

num_full_dict_iapr={
'AS': 0, '2S':1, '3S':2, '4S':3, '5S':4, '6S':5, '7S':6,
'8S':7, '9S':8, '10S':9, 'JS':10, 'QS':11, 'KS':12,
'AH': 0, '2H': 1, '3H': 2, '4H': 3, '5H': 4, '6H': 5,
'7H':6, '8H':7, '9H':8, '10H':9, 'JH': 10, 'QH': 11,
'KH':12,
'AC': 0, '2C': 1, '3C': 2, '4C': 3, '5C': 4, '6C': 5,
'7C':6, '8C':7, '9C':8, '10C':9, 'JC': 10, 'QC': 11,
'KC':12,
'AD': 0, '2D': 1, '3D': 2, '4D': 3, '5D': 4, '6D': 5,
'7D':6, '8D':7, '9D':8, '10D':9, 'JD': 10, 'QD': 11,
'KD':12, 'null':13}
inv_num_dic={'0':'A','1':'2','2':'3','3':'4','4':'5','5':'6','6':'7','7':'8','8':'9','9':'10','10':'J','11':'Q','12':'K','13':'N'}


sb_full_dict_iapr={'D':0,'H':1,'S':2,'C':3,'u':4}
inv_sb_dic = {'0':'D','1':'H','2':'S','3':'C','4':'N'}


In [ ]:
# MLP Model
class MLP(torch.nn.Module):
  def __init__(self,classes=14):
    super(MLP, self).__init__()
    self.fc1 = torch.nn.Linear(3000,500)
    self.fc3 = torch.nn.Linear(500,100)
    self.fc4 = torch.nn.Linear(100,classes)

  def forward(self,input):
    input = torch.nn.functional.relu(self.fc1(input))
    input = self.fc3(input)
    input = self.fc4(input)
    return input

model_num = MLP(classes=14)
path_num = '/content/drive/MyDrive/iapr/Models/num_2022-06-03-06-33.pt'
model_num.load_state_dict(torch.load(path_num))
mn=model_num.eval()
print('mn:',mn)

model_sb = MLP(classes=5)
path_sb = '/content/drive/MyDrive/iapr/Models/sb_2022-06-03-05-51.pt'
model_sb.load_state_dict(torch.load(path_sb))
ms=model_sb.eval()
print('ms:',ms)

In [ ]:
# load raw data
"""
  ims_num: ndarray [Image, Image]
  ims_sb: ndarray [Image, Image]
"""

import pandas as pd
path = '/content/drive/MyDrive/iapr/project/data/train'
df = pd.read_csv(os.path.join(path,'updated_train_labels.csv'),keep_default_na=False) #filelink: https://drive.google.com/file/d/1eVEAf7qZaTpfqygPjPsCIT5nNrsKcMk7/view?usp=sharing

# label for num classification
labelNum=[]
try:
  for i in range(28):
    # pdb.set_trace()
    # for t in ['P1.1','P1.2','P2.1','P2.2','P3.1','P3.2','P4.1','P4.2','T1','T2','T3','T4','T5',]:
    for t in ['P1.1','P1.2','P2.1','P2.2','P3.1','P3.2','P4.1','P4.2']:
      tmp = df.iloc[i]
      labelNum.append(num_full_dict_iapr[tmp[t]])  
  labelNum = np.array(labelNum)
except KeyError:
  print(i)
  print(t)
  print(tmp[t])

# label for symbol classification
labelSb=[]
try:
  for i in range(28):
    for t in ['P1.1','P1.2','P2.1','P2.2','P3.1','P3.2','P4.1','P4.2']:
      tmp = df.iloc[i]
      if len(tmp[t])==3:
        s = tmp[t][2:3]
      else:
        s = tmp[t][1:2]
      labelSb.append(sb_full_dict_iapr[s])
  labelSb = np.array(labelSb)
except KeyError:
  print(i)
  print(t)
  print(tmp[t])


# data for num classification 
ims_num=np.load("/content/drive/MyDrive/iapr/project/data/total_cropped_cards/total_P_digits.npy") # 140,348,228,3 Num,H,W,C

# data for symbol classification
ims_sb=np.load("/content/drive/MyDrive/iapr/project/data/total_cropped_cards/total_P_symbols.npy") # 140,348,228,3 Num,H,W,C


# execute demo


In [ ]:

i=100
im_num=Image.fromarray(ims_num[i].astype(np.uint8)).resize([50,60])
plt.imshow(im_num)
plt.show()
im_num = im_num.convert("L")
im_num = torch.from_numpy(np.array(im_num).flatten().astype(np.float32))

im_sb = Image.fromarray(ims_sb[i].astype(np.uint8)).resize([50,60])
plt.imshow(im_sb)
plt.show()
im_sb = im_sb.convert("L")
im_sb = torch.from_numpy(np.array(im_sb).flatten().astype(np.float32))

outputs_num = model_num(im_num)
_, pred_num = torch.max(outputs_num.data, dim=0)
# print()

outputs_sb = model_sb(im_sb)
_, pred_sb = torch.max(outputs_sb.data, dim=0)
# print(pred_sb.item())
print(inv_num_dic[str(pred_num.item())]+inv_sb_dic[str(pred_sb.item())])



